# Vector boson fusion in IDM

The final notebook is [Significance_countours.ipynb](./Significance_countours.ipynb) which is based on [Sensitivity_fit.ipynb](Sensitivity_fit.ipynb) which is based on [Sensitivity_plots.ipynb](Sensitivity_plots.ipynb) by JD

### Initialization

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import sys
import subprocess
import numpy as np
import pandas as pd
##pip3 install pyslha
#import pyslha # not longer required
import tempfile
import os
import re
def grep(pattern,multilinestring):
    '''Grep replacement in python
    as in: $ echo $multilinestring | grep pattern
    dev: re.M is for multiline strings
    '''
    import re 
    grp=re.finditer('(.*)%s(.*)' %pattern, multilinestring,re.M)
    return '\n'.join([g.group(0) for g in grp])

def subprocess_line_by_line(*args,TRUST_ERRORS=True,**kwargs):
    '''
    Subprocess output line by line. Stop of error found when TRUST_ERRORS=True, and simply
    report wait method otherwise.
    
    The arguments are the same as for the Popen constructor.
    
    WARNING: Works only in Python 3
    
    See: https://stackoverflow.com/a/28319191/2268280 
    and: https://stackoverflow.com/a/17698359/2268280
    
    Example:
    
    subprocess_line_by_line('for i in $(seq 1 3);do echo $i; sleep 1;done',shell=True)
    '''
    
    if not TRUST_ERRORS:
        kwargs['stderr']=subprocess.PIPE
        
    kwargs['stdout']=subprocess.PIPE
    kwargs['bufsize']=1
    kwargs['universal_newlines']=True
    s=subprocess.Popen(*args,**kwargs)
    with s as p:
        for line in p.stdout:
            print(line, end='') # process line here
    
    if TRUST_ERRORS:
        if p.returncode != 0:
            raise subprocess.CalledProcessError(p.returncode, p.args)
    else:
        return s.wait()
    
##Main madGRAPH script:
def preamble(MHc=750,MH0=110,processes='generate p p > e+ e-'):
    return '''import model InertDoublet_UFO
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

'''+processes+'''

output ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL

'''

def lamL_loop(cfg,MHc=750,MH0=110,lamL=0.01):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL
0    
../'''+cfg.LHA_input_file+'''
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL '''+str(lamL)+'''
set mmh0 '''+str(MH0)+'''
set mma0 '''+str(MHc)+'''
set mmhch '''+str(MHc)+'''
0

''' 


def closing(MHc=750,MH0=110):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL -i
print_results --path=./result_BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lamdaL_mh0_110.txt --format=short


done
'''

### Input variables

In [5]:
MHc=750
MH0=240
if MH0%1!=0:
    sys.exit('ERROR: MH0 must be integer')

### Configuration variables

In [6]:
cfg=pd.Series()

In [7]:
cfg['thisroot']='/home/restrepo/prog/ROOT/root/bin/thisroot.sh'
cfg['thisroot']='/opt/root5/bin/thisroot.sh'
cfg['MADGRAPH']='madgraph'
cfg['run_dir']='Task_Asana'
cfg['work_dir']='studies/IDM/'+cfg.run_dir
cfg['work_script']='BP_'+str(int(MHc))+'_A_'+str(int(MH0))+'.txt'
cfg['LHA_input_file']='MadGraph_cards/benchmarks/param_card_template.dat'
cfg['processes']='generate p p > h2 h2 j j @0'
cfg['processes']='generate p p > h2 h2'
cfg['output_dir']='studies/IDM/BP_'+str(int(MHc))+'_'+str(int(MH0))+'_vs_lambdaL' #madGRAPH output

### Running flags

In [8]:
CLONE_GIT_REPO=False #WARNING: Try to overwrite current contents!
INSTALL=True # If True check full installation
TEST=False #take a long time
VERBOSE=True #Print shell commands output line by line
if not INSTALL:
    CLONE_GIT_REPO=False

## Install root 5

### Prerequisites 
```bash
apt-get install cmake git dpkg-dev make g++ gcc binutils libx11-dev libxpm-dev \
libxft-dev libxext-dev gfortran libssl-dev libpcre3-dev \
xlibmesa-glu-dev libglew1.5-dev libftgl-dev \
libmysqlclient-dev libfftw3-dev libcfitsio-dev \
graphviz-dev libavahi-compat-libdnssd-dev \
libldap2-dev python-dev python3-dev libxml2-dev libkrb5-dev \
libgsl0-dev libqt4-dev r-base r-base-dev
```

Install ROOT 5 in some `PATH`
```bash
git clone http://root.cern.ch/git/root.git
cd root
git checkout v5-34-00-patches
./configure
make

```
Add to your `.bashrc`
```bash
source PATH/root/bin/thisroot.sh
```

In [9]:
f=open('kk.sh','w')
f.write('source '+cfg.thisroot+'\n')
f.write('which root\n')
f.close()

if not subprocess.Popen('bash kk.sh'.split(),
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()[0]:
    sys.exit('INSTALL ROOT: see instrucctions in notebook')

# Development of module to calculate one specific point

If `CLONE_GIT_REPO=False`, it is assumed that you did the clone as:
```bash
git clone  --recursive git@github.com:restrepo/VBF_IDM.git
```

In [10]:
if CLONE_GIT_REPO:  
    REPO='VBF_IDM'
    REPO_url='git@github.com:restrepo'
    main_dir='.' #WARNING: Try to overwirte contents
    if os.path.exists(main_dir+'index.ipynb'):
        sys.exit('ERROR: Repo files already exists. Check main_dir')
    if not os.path.isdir(main_dir):
        s=subprocess.Popen(['mkdir','-p',main_dir],stdout=subprocess.PIPE,stderr=subprocess.PIPE).wait()

    td=tempfile.mkdtemp()
    s=subprocess_line_by_line(('git clone  --recursive '+REPO_url+'/'+REPO+'.git').split(),cwd=td,
                 stdout=subprocess.PIPE,stderr=subprocess.PIPE,TRUST_ERRORS=False)

    s=subprocess.Popen('mv '+td+'/'+REPO+'/*  '+main_dir,shell=True,
                   stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    s=subprocess.Popen('mv '+td+'/'+REPO+'/.* '+main_dir,shell=True,
                   stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    os.rmdir(td+'/'+REPO)
    os.rmdir(td)

In [ ]:
if INSTALL:
    s=subprocess.Popen('git checkout -b v2.3.3'.split(),cwd=cfg.MADGRAPH,
                      stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    if 'Switched' not in s[1].decode('utf-8'):
        sys.exit('Submodule problems')

subprocess does not use .bashrc

In [13]:
f=open('madgraph/kk.sh','w')
f.write('source '+cfg.thisroot+'\n')
f.write('./bin/mg5_aMC install.dat\n')
f.close()

In [14]:
if INSTALL:
    if VERBOSE:
        subprocess_line_by_line('bash kk.sh'.split(),cwd='madgraph', TRUST_ERRORS=False )
    else:
        s=subprocess.call('bash kk.sh'.split(),cwd='madgraph', stdout=open('kk','w'),stderr=open('kkk','w') )

************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 2.3.3                 2015-10-25         *
*                                                          *
*    The MadGraph5_aMC@NLO Development Team - Find us at   *
*    https://server06.fynu.ucl.ac.be/projects/madgraph     *
*                       

gfortran -O1 -fno-automatic  -c pyevnt.f
gfortran -O1 -fno-automatic  -c py4jtw.f
gfortran -O1 -fno-automatic  -c pykmap.f
gfortran -O1 -fno-automatic  -c pypdfu.f
gfortran -O1 -fno-automatic  -c pydecy.f
gfortran -O1 -fno-automatic  -c pygdir.f
gfortran -O1 -fno-automatic  -c pysimp.f
gfortran -O1 -fno-automatic  -c pymemx.f
gfortran -O1 -fno-automatic  -c pytime.f
gfortran -O1 -fno-automatic  -c pyinre.f
gfortran -O1 -fno-automatic  -c pysgex.f
gfortran -O1 -fno-automatic  -c py3ent.f
gfortran -O1 -fno-automatic  -c pyadsh.f
gfortran -O1 -fno-automatic  -c pyeicg.f
gfortran -O1 -fno-automatic  -c py4jet.f
gfortran -O1 -fno-automatic  -c pylist.f
gfortran -O1 -fno-automatic  -c pyxxz6.f
gfortran -O1 -fno-automatic  -c pygaus.f
gfortran -O1 -fno-automatic  -c pyrvch.f
gfortran -O1 -fno-automatic  -c pyfint.f
gfortran -O1 -fno-automatic  -c pygvmd.f
gfortran -O1 -fno-automatic  -c pyxdin.f
gfortran -O1 -fno-automatic  -c pykcut.f
gfortran -O1 -fno-automatic  -c pyrvsb.f
gfortran -O1 -fn

ar -cr ./lib/libpgslib.a ./lib/pgslib.o
ranlib ./lib/libpgslib.a
rm ./lib/pgslib.o

----- Now compiling TAUOLA -----

gfortran -fno-automatic -c -o ./lib/tauola.o ./src/tauola.f
ar -cr ./lib/libtauola.a ./lib/tauola.o
ranlib ./lib/libtauola.a
rm ./lib/tauola.o

----- Now compiling STDHEP -----

cd ./src/stdhep-dir; make binlib; make stdhep
make[3]: Entering directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/PGS4/src/stdhep-dir'
test -d ./bin || mkdir -p ./bin
test -d ./lib || mkdir -p ./lib
make[3]: Leaving directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/PGS4/src/stdhep-dir'
make[3]: Entering directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/PGS4/src/stdhep-dir'
(cd src/stdhep; make  all) > log.stdhep 2>&1
make[3]: Leaving directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/PGS4/src/stdhep-dir'
cp ./src/stdhep-dir/lib/libstdhep.a ./lib/libstdhep.a
ranlib ./lib/libstdhep.a

----- Now compiling

make[4]: Leaving directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/lhapdf'
make[3]: Leaving directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/lhapdf'
make[2]: Leaving directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/lhapdf'
cd ../libraries/PGS4; mkdir -p lib; make tauola
make[2]: Entering directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/PGS4'
make[2]: Nothing to be done for 'tauola'.
make[2]: Leaving directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/PGS4'
cd ../libraries/PGS4; mkdir -p lib; make mcfio
make[2]: Entering directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/PGS4'
make[2]: Nothing to be done for 'mcfio'.
make[2]: Leaving directory '/home/restrepo/prog/2017/VBF_IDM/madgraph/pythia-pgs/libraries/PGS4'
gfortran -O -fno-automatic -I../libraries/PGS4/src -o pythia pythia.o ME2pythia.o getjet.o ktclusdble.o pgs_ranmar.o pydata.o -L../libr

>> Compiling external/fastjet/FunctionOfPseudoJet.cc
>> Compiling external/fastjet/GhostedAreaSpec.cc
>> Compiling external/fastjet/JetDefinition.cc
>> Compiling external/fastjet/LazyTiling25.cc
>> Compiling external/fastjet/LazyTiling9.cc
>> Compiling external/fastjet/LazyTiling9Alt.cc
>> Compiling external/fastjet/LazyTiling9SeparateGhosts.cc
>> Compiling external/fastjet/LimitedWarning.cc
>> Compiling external/fastjet/MinHeap.cc
>> Compiling external/fastjet/PseudoJet.cc
>> Compiling external/fastjet/PseudoJetStructureBase.cc
>> Compiling external/fastjet/RangeDefinition.cc
>> Compiling external/fastjet/RectangularGrid.cc
>> Compiling external/fastjet/Selector.cc
>> Compiling external/fastjet/TilingExtent.cc
>> Compiling external/fastjet/Voronoi.cc
>> Compiling external/fastjet/contribs/Nsubjettiness/AxesDefinition.cc
>> Compiling external/fastjet/contribs/Nsubjettiness/ExtraRecombiners.cc
>> Compiling external/fastjet/contribs/Nsubjettiness/MeasureDefinition.cc
>> Compiling externa

In [16]:
if TEST:
    if VERBOSE:
        subprocess_line_by_line('./test.sh'.split(), cwd='test',
            stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    else:
        s=subprocess.Popen('./test.sh'.split(), cwd='test',
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

s[0].decode('utf-8')

### Create MadGraph script with processes

In [17]:
LambdasL=[0.01]#,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9]_*.root | more
delphes_events_70_001_.root
delphes_events_70_002_.root
delphes_events_70_003_.root
...
...
delphes_events_90_018_.root
delphes_events_90_019_.root
delphes_events_90_020_.root
```

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9][0-9]_*.root | more
delphes_events_110_001_.root
delphes_events_110_002_.root
delphes_events_110_003_.root
...
...
delphes_events_240_018_.root
delphes_events_240_019_.root
delphes_events_240_020_.root
```

cat readme.txt 
date 23/01/2017
Delphes output (.root) for the set the points contained in data_mh0_vs_x.dat

The data is organized as follows (in ascending order in lambda_L[lal] ):

#lal #mh0[GeV] #xs[fb]   #Delphes name
0.3 63.75 200        --> delphes_events_2.root
0.4 64.80 200        --> delphes_events_3.root 
0.5 66.32 200        --> delphes_events_4.root
0.6 68.05 200        --> delphes_events_5.root 
0.7 70.04 200        --> delphes_events_6.root
.   .     .          --> 
.   .     .          --> 
.   .     .          -->

The cross-section (xs) for the processes p p > h0 h0 j j
for all the set of points is around 200 fb. [Except for
the first point, with lal=0.3,  where the cross-section turns out to be 179 fb.]

In [18]:
s=subprocess.Popen(['mkdir','-p',cfg.work_dir],
                 stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

In [19]:
f=open(cfg.work_dir+'/'+cfg.work_script,'w')
f.write( preamble(MHc,MH0,cfg.processes) )
for lamL in LambdasL:
    f.write( lamL_loop(cfg,MHc,MH0,lamL=lamL) )
f.write( closing(MHc,MH0) )
f.close()

```bash
# pip3 install subprocess.run && #pip2 install subprocess.run 
```

In [20]:
f=open(cfg.MADGRAPH+'/kk.sh','w')
f.write('source '+cfg.thisroot+'\n')
f.write('./bin/mg5_aMC ../'+cfg.work_dir+'/'+cfg.work_script+'\n')
f.close()

In [21]:
if not VERBOSE:
    s=subprocess.Popen( 'bash kk.sh'.split(), cwd=cfg.MADGRAPH,
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    s.wait()

In [22]:
if VERBOSE:
    subprocess_line_by_line( 'bash kk.sh'.split(), cwd=cfg.MADGRAPH,TRUST_ERRORS=False)

************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 2.3.3                 2015-10-25         *
*                                                          *
*    The MadGraph5_aMC@NLO Development Team - Find us at   *
*    https://server06.fynu.ucl.ac.be/projects/madgraph     *
*                       

INFO: reload from .py file 
INFO: load particles 
INFO: load vertices 
INFO: Change particles name to pass to MG5 convention 
************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 2.3.3                 2015-10-25         *
*                                                          *
*    The MadGraph5_a

%%bash
cd madgraph/
./bin/mg5_aMC ../studies/IDM/Task_Asana/BP_750_A_110.txt
cd ..

### Prepare madevent script

In [23]:
pythia_script='TemplateRunPythiaDelphes_all.dat'
if len(LambdasL)>99:
    sys.exit('ERROR: UPDATE FORMAT FOR > 99 runs')
f=open(cfg.work_dir+'/'+pythia_script,'w')
for r in range(1,len(LambdasL)+1):
    f.write('pythia run_%02d\n' %r)
    f.write('3\n')
    f.write('0\n')
f.close()

In [24]:
f=open(cfg.output_dir+'/kk.sh','w')
f.write('source '+cfg.thisroot+'\n')
f.write('./bin/madevent ../'+cfg.run_dir+'/'+pythia_script+'\n')
f.close()

In [25]:
#if not VERBOSE:
s=subprocess.Popen('bash kk.sh'.split(), cwd=cfg.output_dir,
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)

s.wait()
(PHOUT,PHERR)=s.communicate()

print(PHOUT.decode('utf-8'))

In [28]:
cs_pb=np.array( re.sub( '\s+\+\-\s+[0-9\+\-eE\.]+\s+pb','\n',  
          re.sub('\s+Cross-section\s+:\s+','' ,
          ''.join( grep('Cross-section',PHOUT.decode('utf-8')).split('\n') 
          ) )  ).strip().split('\n')  ).astype(float64)

In [29]:
if len(cs_pb)==len(LambdasL):
    df=pd.DataFrame({'xs_'+str(int(MH0)):cs_pb,'laL':LambdasL})
else:
    sys.exit('Error: missing cross section')

In [30]:
df

,laL,xs_240
0,0.01,5.288000e-08


In [40]:
from nose.tools import assert_equal
def test_all(df):
    assert_equal(df.xs_240.values[0],5.288E-8)
    
    

In [23]:
df.to_csv('output/cs_'+str(int(MHc))+'_'+str(int(MH0))+'.csv',index=False)

### Get results

In [33]:
full_output='output'
s=subprocess.Popen(['mkdir', '-p',full_output]).wait()

#if 'xs_'+str(int(MH0)) not in df.columns.values:
if True:
    for r in range(1,len(LambdasL)+1):
        nrun='%02d' %r
        nrun3='%03d' %r
        s=subprocess.Popen(['cp',cfg.output_dir+'/Events/run_'+nrun+'/tag_1_delphes_events.root', 
                            full_output+'/delphes_events_'+str(int(MH0))+'_'+nrun3+'_.root'],
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if s.wait()>0:
            sys.exit('Files not found')

ls studies/IDM/BP_750_110_vs_lambdaL/Events/run_07/

### Merge masses

In [24]:
df_full=pd.read_csv('Output_data.csv')

In [25]:
dff=df_full.merge(df,on='laL',how='left').fillna(0)

In [26]:
tmp=dff['laL']
dff=dff.drop('laL',axis='columns')
dff['laL']=tmp

In [27]:
dff

,Unnamed: 0,xs_70,xs_75,xs_80,xs_85,xs_90,xs_110,xs_130,xs_150,xs_170,xs_190,xs_210,xs_220,xs_240_x,xs_240_y,laL
0,0,0.007836,0.007475,0.007075,0.006754,0.006451,0.005417,0.004505,0.003759,0.003205,0.002678,0.002277,0.002097,0.001752,0.001752,0.01
1,1,0.008207,0.007771,0.007350,0.006937,0.006623,0.005523,0.004599,0.003848,0.003219,0.002666,0.002247,0.002064,0.001755,0.001755,0.02
2,2,0.009770,0.008817,0.008136,0.007636,0.007198,0.005774,0.004747,0.003979,0.003283,0.002781,0.002315,0.002109,0.001776,0.001776,0.05
3,3,0.011421,0.009917,0.008990,0.008275,0.007629,0.006041,0.004918,0.004074,0.003398,0.002825,0.002356,0.002160,0.001804,0.001804,0.07
4,4,0.014126,0.011538,0.010077,0.009139,0.008302,0.006406,0.005106,0.004190,0.003460,0.002875,0.002400,0.002185,0.001843,0.001843,0.10
5,5,0.019924,0.015071,0.012550,0.010998,0.009767,0.007094,0.005427,0.004425,0.003637,0.002974,0.002492,0.002259,0.001883,0.001883,0.15
6,6,0.027188,0.019742,0.015811,0.013316,0.011713,0.007997,0.006022,0.004730,0.003859,0.003159,0.002543,0.002340,0.001944,0.001944,0.20
7,7,0.036839,0.025211,0.019486,0.016115,0.013757,0.008816,0.006455,0.005012,0.004055,0.003234,0.002655,0.002430,0.001983,0.001983,0.25
8,8,0.047742,0.031698,0.023874,0.019238,0.016198,0.009981,0.007122,0.005388,0.004225,0.003419,0.002787,0.002509,0.002075,0.002075,0.30
9,9,0.059759,0.039166,0.028822,0.022678,0.018853,0.011133,0.007755,0.005748,0.004519,0.003553,0.002896,0.002618,0.002111,0.002111,0.35
